In [1]:
import os
import cv2


DATA_DIR = './data1'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

number_of_classes = 10
dataset_size = 100

cap = cv2.VideoCapture(0)

for j in range(number_of_classes):
    if not os.path.exists(os.path.join(DATA_DIR, str(j))):
        os.makedirs(os.path.join(DATA_DIR, str(j)))

    print('Collecting data for class {}'.format(j))

    done = False
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, 'Ready? Press "Q" ! :)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(DATA_DIR, str(j), '{}.jpg'.format(counter)), frame)

        counter += 1
        
while True:
    ret, frame = cap.read()
    cv2.putText(frame, 'Ready? Press "Esc" to exit!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) == 27:  # ASCII value of 'Esc' key is 27
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
import os
import pickle

import mediapipe as mp
import cv2
import matplotlib.pyplot as plt


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = './data1'

data = []
labels = []
for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []

        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            data.append(data_aux)
            labels.append(dir_)

            

f = open('data1.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()

In [1]:
print(data)

NameError: name 'data' is not defined

In [2]:
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import numpy as np


data_dict = pickle.load(open('./data1.pickle', 'rb'))

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

model_digit = RandomForestClassifier(max_depth=5, random_state=0)

model_digit.fit(x_train, y_train)

y_predict = model_digit.predict(x_test)
test_accuracy = accuracy_score(y_test, y_predict)
print('Test Accuracy: {:.2f}%'.format(test_accuracy * 100))

# Perform cross-validation to assess generalization performance
cv_scores = cross_val_score(model_digit, data, labels, cv=5)
print('Cross-Validation Scores: {}'.format(cv_scores))
print('Mean CV Accuracy: {:.2f}%'.format(np.mean(cv_scores) * 100))

f = open('model_trial.p', 'wb')
pickle.dump({'model': model_digit}, f)
f.close()

Test Accuracy: 100.00%
Cross-Validation Scores: [0.98994975 0.9798995  1.         1.         1.        ]
Mean CV Accuracy: 99.40%


In [3]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# Confusion Matrix
cm = confusion_matrix(y_test, y_predict)
 
# Precision
precision = precision_score(y_test, y_predict, average='weighted')
print("Precision: ", precision)

# Recall
recall = recall_score(y_test, y_predict, average='weighted')
print("Recall: ", recall) 

# F1 Score
f1 = f1_score(y_test, y_predict, average='weighted')
print("F1 Score: ", f1)



# import seaborn as sns
# sns.heatmap(cm, annot=True)
print("Accuracy:", accuracy_score(y_test,y_predict)) 


Precision:  1.0
Recall:  1.0
F1 Score:  1.0
Accuracy: 1.0


In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk

# Load the hand gesture recognition model
model_digit_dict = pickle.load(open('./model_trial.p', 'rb'))
model_digit = model_digit_dict['model']

root = tk.Tk()
root.title("Hand Gesture Recognition")
root.geometry("1100x1000")

# Create a canvas to display the webcam feed
canvas = tk.Canvas(root, width=600, height=600)
canvas.pack()

predicted_label = tk.Label(root, text="Predicted Character: ", font=("Helvetica", 16))
predicted_label.pack()

save_text = tk.Label(root, text="Word: ", font=("Helvetica", 16))
save_text.pack()


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Define labels for hand gestures
labels_dict_digit = {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '0'}

# Function to process webcam frames and perform hand gesture recognition

def process_frame():
    ret, frame = cap.read()
    if ret:
        H, W, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(frame_rgb)
        if results.multi_hand_landmarks:
            # Use only the first detected hand (index 0)
            hand_landmarks = results.multi_hand_landmarks[0]

            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            data_aux = []
            x_ = []
            y_ = []

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10

            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            prediction = model_digit.predict([np.asarray(data_aux)])
            predicted_character = labels_dict_digit[int(prediction[0])]
            predicted_label.config(text="Predicted Character: " + predicted_character)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                        cv2.LINE_AA)

        # Convert the frame to a format that can be displayed by Tkinter
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img = ImageTk.PhotoImage(image=img)
        canvas.create_image(0, 0, anchor=tk.NW, image=img)
        canvas.img = img

    # Call this function again after a delay
    root.after(30, process_frame)
word = "" 
def save_letter():
    global word
    predicted_character = predicted_label.cget("text").split()[-1]  # Get the predicted character from the label
    word+=predicted_character
    if len(word) > 0:  # Save the letter if it's a single character 
        save_text.config(text="Word: " + word)
       
def save_word():
    global word
    if len(word) > 0:
        # Save the entire word to the file and reset the word variable
        with open("letters.txt", "a") as f:
            f.write(word + "\n")  # Write the word to the file
        
        # Reset the word variable and update the save_text label
        word = ""
        save_text.config(text="Word: " + word)

def clear():
    save_text.config(text="Word: " )
    global word
    word=""
    
# Start capturing from the webcam
cap = cv2.VideoCapture(0)

# Call the function to start processing frames
process_frame()

# Button to save the predicted character
save_button = tk.Button(root, text="Save Letter", command=save_letter,bd=2, padx=10, pady=5)
save_button.pack()

clear_button = tk.Button(root,text="clear",command=clear,bd=2, padx=10, pady=5)
clear_button.pack()

# Run the Tkinter event loop
root.mainloop()

# Release the video capture and close all windows when done
cap.release()
cv2.destroyAllWindows()

In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk

# Load the hand gesture recognition model
model_digit_dict = pickle.load(open('./model_trial.p', 'rb'))
model_digit = model_digit_dict['model']

model_hand_dict = pickle.load(open('./model.p', 'rb'))
model_hand = model_hand_dict['model']  


root = tk.Tk()
root.title("Hand Gesture Recognition")
root.geometry("1100x1000")

# Create a canvas to display the webcam feed
canvas = tk.Canvas(root, width=600, height=600)
canvas.pack()

predicted_label = tk.Label(root, text="Predicted Character: ", font=("Helvetica", 16))
predicted_label.pack()

mode_label = tk.Label(root, text="Current Mode: Hand Gesture Recognition", font=("Helvetica", 14))
mode_label.pack()

save_text = tk.Label(root, text="Word: ", font=("Helvetica", 16))
save_text.pack()


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Define labels for hand gestures
labels_dict_digit = {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '0'}

labels_dict_hand = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'K', 10: 'L', 11: 'M',
               12: 'N', 13: 'O', 14: 'Q', 15: 'R', 16: 'S', 17: 'T', 18: 'U', 19: 'V', 20: 'W', 21: 'X', 22: 'Y',
               23: 'THUMBS_UP', 24: 'HEART', 25: 'SUPER'}


def process_hand_frame(frame):
        H, W, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(frame_rgb)
        if results.multi_hand_landmarks:
            # Use only the first detected hand (index 0)
            hand_landmarks = results.multi_hand_landmarks[0]

            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            data_aux = []
            x_ = []
            y_ = []

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10

            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            prediction = model_hand.predict([np.asarray(data_aux)])
            predicted_character = labels_dict_hand[int(prediction[0])]
            predicted_label.config(text="Predicted Character: " + predicted_character)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                        cv2.LINE_AA)
# Function to process webcam frames and perform hand gesture recognition

def process_digit_frame(frame):
        H, W, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(frame_rgb)
        if results.multi_hand_landmarks:
            # Use only the first detected hand (index 0)
            hand_landmarks = results.multi_hand_landmarks[0]

            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            data_aux = []
            x_ = []
            y_ = []

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10

            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            prediction = model_digit.predict([np.asarray(data_aux)])
            predicted_character = labels_dict_digit[int(prediction[0])]
            predicted_label.config(text="Predicted Character: " + predicted_character)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                        cv2.LINE_AA)

       


def process_frame():
    ret, frame = cap.read()
    if ret:
        if mode == "hand":
            process_hand_frame(frame)
        elif mode == "digit":
            process_digit_frame(frame)

        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img = ImageTk.PhotoImage(image=img)
        canvas.create_image(0, 0, anchor=tk.NW, image=img)
        canvas.img = img

    # Call this function again after a delay
    root.after(30, process_frame)

def switch_to_hand_mode():
    global mode
    mode = "hand"
    mode_label.config(text="Current Mode: Hand Gesture Recognition")

def switch_to_digit_mode():
    global mode
    mode = "digit"
    mode_label.config(text="Current Mode: Digit Recognition")

mode = "hand"  # Start with hand gesture recognition mode

hand_mode_button = tk.Button(root, text="Hand Gesture Recognition", command=switch_to_hand_mode, bd=2, padx=10, pady=5)
hand_mode_button.pack(side=tk.LEFT, padx=20, pady=10)

digit_mode_button = tk.Button(root, text="Digit Recognition", command=switch_to_digit_mode, bd=2, padx=10, pady=5)
digit_mode_button.pack(side=tk.LEFT, padx=20, pady=10)

word = "" 
def save_letter():
    global word
    predicted_character = predicted_label.cget("text").split()[-1]  # Get the predicted character from the label
    word+=predicted_character
    if len(word) > 0:  # Save the letter if it's a single character 
        save_text.config(text="Word: " + word)
       
def save_word():
    global word
    if len(word) > 0:
        # Save the entire word to the file and reset the word variable
        with open("letters.txt", "a") as f:
            f.write(word + "\n")  # Write the word to the file
        
        # Reset the word variable and update the save_text label
        word = ""
        save_text.config(text="Word: " + word)

def clear():
    save_text.config(text="Word: " )
    global word
    word=""
    
# Start capturing from the webcam
cap = cv2.VideoCapture(0)

# Call the function to start processing frames
process_frame()

# Button to save the predicted character
save_button = tk.Button(root, text="Save Letter", command=save_letter,bd=2, padx=10, pady=5)
save_button.pack()

clear_button = tk.Button(root,text="clear",command=clear,bd=2, padx=10, pady=5)
clear_button.pack()

# Run the Tkinter event loop
root.mainloop()

# Release the video capture and close all windows when done
cap.release()
cv2.destroyAllWindows()

In [15]:
import numpy as np

def multiple_regression_model(x,y,learning_rate=0.01,num_interatons=1000):
  n, m=x.shape
  x=np.hstack((np.ones((n,1)),x))
  weights=np.zeros((m+1,1))
  for _ in range(num_interatons):
    predict=x @ weights
    loss=((1/2*n))*np.sum((predict-y)**2)
    gradient=(1/n) * x.T @ (predict-y)
    weights -= learning_rate*np.mean(gradient,axis=1,keepdims=True)
  def predict(x):
    x_with_bias=np.hstack((1,x))
    return np.dot(x_with_bais, weights).item()
    
    return predict

x_new=np.array([[1,2],[2,3],[3,4],[4,5]])
y_new=np.array([4,7,10,13])
multiple_regression_model_new= multiple_regression_model(x_new ,y_new,learning_rate=0.01,num_interatons=1000)
print(multiple_regression_model_new)

for sample in x_new :
   prediction = multiple_linear_regression_model_new ( sample)
   print ( f" Input : { sample } , Prediction : { prediction }")


None


NameError: name 'multiple_linear_regression_model_new' is not defined

In [32]:
import numpy as np

def multiple_linear_regression (X , y , learning_rate =0.01 ,
num_iterations =1000) :
    N , M = X . shape

    X = np . hstack (( np . ones (( N , 1) ) , X ) )
    weights = np . zeros (( M + 1 , 1) )

    for _ in range ( num_iterations ) :
        predictions = X @ weights
        loss = (1 / (2 * N ) ) * np . sum (( predictions - y ) **2)
        gradients = (1 / N ) * X . T @ ( predictions - y )
        weights -= learning_rate * np . mean ( gradients , axis=1 , keepdims = True )

    def predict ( x ) :
        x_with_bias = np . hstack ((1 , x ) )
        return np . dot ( x_with_bias , weights ) . item ()

    return predict

 # Example usage :

X_new = np . array ([[1 , 2] , [2 , 3] , [3 , 4] , [4 , 5]])
y_new = np . array ([4 , 7 , 10 , 13])

multiple_linear_regression_model_new =multiple_linear_regression ( X_new , y_new , learning_rate=0.01 , num_iterations =1000)

for sample in X_new :
    prediction = multiple_linear_regression_model_new ( sample)
    print ( f"Input : { sample } , Prediction : { prediction }")


Input : [1 2] , Prediction : 7.703380342345878
Input : [2 3] , Prediction : 8.12531112521953
Input : [3 4] , Prediction : 8.547241908093184
Input : [4 5] , Prediction : 8.969172690966836


In [28]:
import math

 # Sigmoid function
def sigmoid ( z ):
    return 1 / (1 + math . exp ( - z ))

 # Hypothesis function
def hypothesis ( theta , X ):
    return sigmoid ( sum ([ theta [ i ] * X [ i ]
    for i in range ( len ( theta ))]))

 # Cost function
def cost_function ( theta , X , y ):
    m = len ( y )
    total_cost = 0
    for i in range ( m ):
        h = hypothesis ( theta , X [ i ])
        total_cost += ( - y [ i ] *
        math . log ( h )) -((1 - y [ i ]) *math . log (1 - h ))
    return total_cost / m

 # Gradient descent
def gradient_descent ( theta , X , y , alpha , num_iters ):
    m = len ( y )
    theta_updated = theta . copy ()
    for _ in range ( num_iters ):
        gradients = [0] * len ( theta )
    for i in range ( len ( theta )):
        sum = 0
    for j in range ( m ):
        h = hypothesis ( theta_updated , X [ j ])
        sum += ( h - y [ j ]) * X [ j ][ i ]
        gradients [ i ] = sum / m
    for i in range ( len ( theta )):
        theta_updated [ i ] = theta_updated [ i ]- alpha* gradients [ i ]
    return theta_updated


 # Predict function
def predict ( theta , X ):
    predictions = []
    for x in X :
        h = hypothesis ( theta , x )
        prediction = 1 if h >= 0.5 else 0
        predictions . append ( prediction )
    return predictions

 # Example usage
X = [[1 , 35] , [1 , 25] , [1 , 10] , [1 , 15] , [1 , 45]]
y = [1 , 0 , 0 , 0 , 1]
theta = [0 , 0]
alpha = 0.1
num_iters = 10000

updated_theta = gradient_descent ( theta , X , y , alpha , num_iters )
print ( " Updated ␣ theta ␣ values : " , updated_theta )
predictions = predict ( updated_theta , X )
print ( " Predictions : " , predictions )


 Updated ␣ theta ␣ values :  [0.0, 0.30000000000000004]
 Predictions :  [1, 1, 1, 1, 1]
